# Langchain Basics

## Initial Settings
### Load Libraries

In [30]:
import os
import openai
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent

import pandas as pd
from dotenv import load_dotenv

### Set Environment Variables for Azure OpenAI Access

In [11]:
load_dotenv()
API_KEY = os.getenv("apikey")
BASE_ENDPOINT = os.getenv("api_base")
COMPLETION_MODEL = os.getenv("completion_model")
CHAT_MODEL = os.getenv("chat_model")
EMBEDDING_MODEL = os.getenv("embedding_model")
DEPLOYMENT_VERSION = os.getenv("api_version")

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = DEPLOYMENT_VERSION
openai.api_base = BASE_ENDPOINT
openai.api_key = API_KEY

### Initalizing Azure OpenAI Model Object

In [3]:
# Initializing Davinci Model for text completion 
llm = AzureOpenAI(
    deployment_name=COMPLETION_MODEL ,
    openai_api_key= API_KEY ,openai_api_version =DEPLOYMENT_VERSION
    )

In [4]:
# Use this format for Chatgpt or GPT4
gptchat = AzureChatOpenAI(
    deployment_name=CHAT_MODEL,
    openai_api_base = BASE_ENDPOINT,
    openai_api_key= API_KEY,
    openai_api_version="2023-03-15-preview"
)

### Simple Prompt Test

In [6]:
joke = llm("Tell me a dad joke")
print(joke)



Q: What did the baby corn say to the mama corn?
A: Where's my popcorn?


### Simple Chat Test



In [9]:
response = gptchat(
    [
        SystemMessage(content="You are an AI assistant that helps a user with marketing content. Keep responses short and under 5 sentences."),
        HumanMessage(content="What are top movie trends in US")
    ]
)
response

AIMessage(content='As of 2021, top movie trends in the US include:\n\n1. Streaming service releases: With COVID-19 restrictions, more movies are premiering on streaming platforms like Netflix, Hulu, and Disney+.\n2. Superhero movies: The Marvel and DC cinematic universes continue to dominate the box office with films like Black Widow and The Suicide Squad.\n3. Nostalgia and remakes: Revivals of classic franchises such as Ghostbusters: Afterlife and The Matrix Resurrections are popular.\n4. Diverse representation: Audiences are demanding more diversity, with films like In the Heights and Shang-Chi and the Legend of the Ten Rings showcasing underrepresented cultures.\n5. High-concept sci-fi: Mind-bending, visually-stunning films like Dune and Tenet are gaining traction as audiences crave immersive experiences.', additional_kwargs={}, example=False)

### Embeddings

In [10]:
openai_embed = OpenAIEmbeddings(model=EMBEDDING_MODEL,
                                openai_api_key=API_KEY)

query_embeddings = openai_embed.embed_query("The sky is blue") 

print(type(query_embeddings))
print(len(query_embeddings))
print(query_embeddings[:5])

<class 'list'>
1536
[0.009326309897005558, -0.0006310197059065104, 0.006163125857710838, -0.015400114469230175, -0.013527460396289825]


### Using Prompt Templates for use input and capturing any variables / non static informations

In [12]:
# Define a prompt template with placeholders for input variables.
prompt_template = PromptTemplate(
        template = 'Write a small advertisement a new {product} and its description {product_desc}?',
        input_variables = ['product', 'product_desc']
)

# Create the final prompt by filling in the defined variables.
final_prompt = prompt_template.format(product='Headphones',  product_desc='Best headphones')
print("FINAL PROMPT: " + final_prompt)

response = llm(final_prompt) 
print(response)

FINAL PROMPT: Write a small advertisement a new Headphones and its description Best headphones?
 Look no further!

Introducing the newest headphones on the market: the XOXO headphones. These headphones offer superior sound quality, with a deep bass and clear treble that will make your music sound truly amazing. The headphones come with a comfortable, adjustable headband for a snug fit and soft ear pads for added comfort. The noise isolation technology ensures your music will remain clear, even in noisy environments. With a sleek, modern design, these headphones are perfect for any occasion. Get your XOXO headphones today and experience the best sound quality money can buy!


In [13]:
# Reuse the prompt Template
print(llm(prompt_template.format(product ='Electric Vehicle', product_desc='Iceland')))



Introducing the new Electric Vehicle from Iceland! Our Electric Vehicle is an eco-friendly car that is powered by a lithium-ion battery, providing a convenient and affordable way to travel. With a range of up to 200 kilometers, you can explore the great outdoors of Iceland without having to worry about refueling. The Electric Vehicle is also equipped with a range of safety features, including advanced airbags, traction control, and an anti-lock braking system, to ensure a safe and secure ride. Come experience the power and convenience of the new Electric Vehicle from Iceland!


### Structured Output Parser

In [14]:
# 1. Define the schema

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="product", description="This is a product"),
    ResponseSchema(name="product_desc", description="This is a short description of the product")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [15]:
# 2. Generates output format instructions which can be included in your prompt.

format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"product": string  // This is a product
	"product_desc": string  // This is a short description of the product
}
```


In [16]:
# 3. Incorporate into final prompt.

template = '''
You will be given a product by the user.
Please provide a imaginary, creative and non offensive product name and a short description of the product.

{format_instructions}

% PRODUCT:
{product}

YOUR RESPONSE:
'''

prompt_template = PromptTemplate(
    input_variables=["product"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

prompt = prompt_template.format(product="Iphone")
print('FINAL PROMPT:')
print(prompt)

FINAL PROMPT:

You will be given a product by the user.
Please provide a imaginary, creative and non offensive product name and a short description of the product.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"product": string  // This is a product
	"product_desc": string  // This is a short description of the product
}
```

% PRODUCT:
Iphone

YOUR RESPONSE:



In [21]:
# 4. LLM now outputs an easily parsable response.

response = llm(prompt)
print(response)

```json
{
	"product": "Iphone",
	"product_desc": "A sleek and powerful phone with a modern design and the latest technology features."
}
```


In [23]:
# 5. Output parser can extract named fields in the schema.

output_parser.parse(response)['product_desc']

'A sleek and powerful phone with a modern design and the latest technology features.'

### Sequential Chains

In [24]:
# Chain component 1: 

from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

template1 = """You will be given an error from the user. Please provide a possible solution.

% ERROR
{error}

YOUR RESPONSE:
"""
prompt_template1 = PromptTemplate(input_variables=["error"], template=template1)

# LLMChain is a simple chain which takes a prompt template, formats with user input and returns LLM output.
error_chain = LLMChain(llm=llm, prompt=prompt_template1)

In [25]:
# Chain component 2: 

template2 = """Given the error message, provide a short solution for fixing it.

% MESSAGE
{message}

YOUR RESPONSE:
"""
prompt_template2 = PromptTemplate(input_variables=["message"], template=template2)

message_chain = LLMChain(llm=llm, prompt=prompt_template2)

In [26]:
# Run the chain
overall_chain = SimpleSequentialChain(
    chains=[error_chain, message_chain],
    verbose=True)
chain_output = overall_chain.run("ZeroDivisionError")



> Entering new  chain...
This error occurs when attempting to divide by zero. To fix this, make sure that you are not attempting to divide by zero, or if you are, use a conditional statement to handle the error.
To fix this error, make sure that you are not attempting to divide by zero. If you are, use a conditional statement to handle the error and prevent it from occurring.

> Finished chain.


In [27]:
print(chain_output)

To fix this error, make sure that you are not attempting to divide by zero. If you are, use a conditional statement to handle the error and prevent it from occurring.


### Document loader & text splitting**

In [28]:
from langchain.document_loaders import GutenbergLoader

# Load a long document.
loader = GutenbergLoader('https://www.gutenberg.org/cache/epub/1515/pg1515.txt')
doc = loader.load()

print(doc[0].page_content[:80].rstrip())
print('Document length in characters:', len(doc[0].page_content))

The Project Gutenberg eBook of The Merchant of Venice, by William Shakespeare
Document length in characters: 154162


In [29]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# We need to split long documents into smaller chunks to stay under the LLM token limit.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(doc[0].page_content)
texts[2]

'THE PRINCE OF MOROCCO, suitor to Portia\r\n\n\nTHE PRINCE OF ARRAGON, suitor to Portia\r\n\n\nANTONIO, a merchant of Venice\r\n\n\nBASSANIO, his friend, suitor to Portia\r\n\n\nGRATIANO, friend to Antonio and Bassanio\r\n\n\nSOLANIO, friend to Antonio and Bassanio\r\n\n\nSALARINO, friend to Antonio and Bassanio\r\n\n\nLORENZO, in love with Jessica\r\n\n\nSHYLOCK, a rich Jew\r\n\n\nTUBAL, a Jew, his friend\r\n\n\nLAUNCELET GOBBO, a clown, servant to Shylock\r\n\n\nOLD GOBBO, father to Launcelet\r\n\n\nLEONARDO, servant to Bassanio\r\n\n\nBALTHAZAR, servant to Portia\r\n\n\nSTEPHANO, servant to Portia\r\n\n\nSALERIO, a messenger from Venice\r\n\n\n\r\n\n\nPORTIA, a rich heiress\r\n\n\nNERISSA, her waiting-woman\r\n\n\nJESSICA, daughter to Shylock\r\n\n\n\r\n\n\nMagnificoes of Venice, Officers of the Court of Justice, a Gaoler,\r\n\n\nServants and other Attendants\r\n\n\n\r\n\n\nSCENE: Partly at Venice, and partly at Belmont, the seat of Portia on\r\n\n\nthe Continent\r\n\n\n\r\n\n\n\r\n

### Agents
#### CSV Agent Example

In [36]:
agent = create_csv_agent(llm, 'goalies17.csv', verbose=True)

In [37]:
agent.run("Who saved the most goals and how many total goals did they save?")



> Entering new  chain...
Thought: I should look at the 'Saves' column in the dataframe
Action: python_repl_ast
Action Input: df['Saves'].max()
Observation: 28928.0
Thought: I should also look at the player who saved the most goals
Action: python_repl_ast
Action Input: df[df['Saves'] == df['Saves'].max()]
Observation:              Player  First  Last  Active  Games Played  Games Started    Win  \
392  Martin Brodeur   1991  2015      22          1266          369.0  691.0   

      Loss  Ties and Overtime Losses  Goals Against  Shot Saved    Saves  \
392  397.0                     154.0           2781     31709.0  28928.0   

     Save Percentage  Goals Against Average  Shutouts  Penalties In Minutes  \
392            0.912                   2.24       125                   122   

     Minutes  
392    74439  
Thought: I now know the final answer
Final Answer: Martin Brodeur saved 28928 goals.

> Finished chain.


'Martin Brodeur saved 28928 goals.'

In [6]:


conversation = ConversationChain(
    llm=gptchat,
    memory=ConversationBufferMemory()
)

conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")
# -> The first three colors of a rainbow are red, orange, and yellow.
conversation.run("And the next 4?")
# -> The next four colors of a rainbow are green, blue, indigo, and violet.

'Green, blue, indigo, violet.'

In [14]:
template = """You are a article generation tool that generates articles for\
            AXE Deordrant. Use title similar to Number of Tips from Pros to Make Your Fragrance Last and Keep Your Scent Game Strong. Keywords to include: how to make cologne last longer, long-lasting deodorant , how to make deodorant last longer , how to make fragrance last longer, long lasting deodorant spray, how to layer fragrances, long lasting body spray, how long does body spray last, how long do fragrances last, how to apply fragrance, how to make fragrance last longer, where to apply fragrance, long lasting body spray for men, best fragrance layering combinations, long lasting men's fragrance, deodorant , deodorant for men, body spray, antiperspirant, antiperspirant, mens body spray, deodorant body spray.\
            The content should use details keywords and explain and extend them and use {content}. \
            The word count of the article should be minimum 1000 words
            {chat_history}
            Human: {content}
            Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "content"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [16]:
llm_chain = LLMChain(
    llm=gptchat,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [18]:
intro_text = llm_chain.predict(content= "Write an introductory paragraph that gives context to the reader, introducing the overall theme of the article. Try to include the target keywords")



> Entering new  chain...
Prompt after formatting:
You are a article generation tool that generates articles for            AXE Deordrant. Use title similar to Number of Tips from Pros to Make Your Fragrance Last and Keep Your Scent Game Strong. Keywords to include: how to make cologne last longer, long-lasting deodorant , how to make deodorant last longer , how to make fragrance last longer, long lasting deodorant spray, how to layer fragrances, long lasting body spray, how long does body spray last, how long do fragrances last, how to apply fragrance, how to make fragrance last longer, where to apply fragrance, long lasting body spray for men, best fragrance layering combinations, long lasting men's fragrance, deodorant , deodorant for men, body spray, antiperspirant, antiperspirant, mens body spray, deodorant body spray.            The content should use details keywords and explain and extend them and use Write an introductory paragraph that gives context to the reader, introducin

In [19]:
intro_text

"Mastering the art of a captivating and long-lasting scent is an essential aspect of men's grooming and personal style. In this comprehensive guide, we delve into expert tips on how to make cologne and deodorant last longer, ensuring that your scent game remains strong throughout the day. We'll discuss the ins and outs of long-lasting deodorant sprays, fragrance layering techniques, and the best application methods for various products, ranging from antiperspirants to men's body sprays. Learn about the longevity of fragrances, where to apply them for maximum impact, and the ideal fragrance combinations to create a signature scent that is uniquely you. Get ready to elevate your fragrance game with this in-depth exploration of deodorants, antiperspirants, and body sprays for men."

In [20]:
content_1 = llm_chain.predict(content= "Discuss how fragrance should be applied on clean, dry, moisturized skin, why having clean, well-hydrated skin aids in retaining fragrance")



> Entering new  chain...
Prompt after formatting:
You are a article generation tool that generates articles for            AXE Deordrant. Use title similar to Number of Tips from Pros to Make Your Fragrance Last and Keep Your Scent Game Strong. Keywords to include: how to make cologne last longer, long-lasting deodorant , how to make deodorant last longer , how to make fragrance last longer, long lasting deodorant spray, how to layer fragrances, long lasting body spray, how long does body spray last, how long do fragrances last, how to apply fragrance, how to make fragrance last longer, where to apply fragrance, long lasting body spray for men, best fragrance layering combinations, long lasting men's fragrance, deodorant , deodorant for men, body spray, antiperspirant, antiperspirant, mens body spray, deodorant body spray.            The content should use details keywords and explain and extend them and use Discuss how fragrance should be applied on clean, dry, moisturized skin, why

In [21]:
content_1

"To achieve the best results and ensure that your fragrance lasts throughout the day, it's crucial to apply it on clean, dry, and moisturized skin. By following these simple guidelines, you can significantly enhance the longevity and effectiveness of your chosen scent.\n\nFirst and foremost, clean skin provides the ideal canvas for your fragrance. When you shower and cleanse your skin, you remove dirt, oils, and any remnants of previously applied fragrances, allowing the new scent to adhere better to your skin. Additionally, a clean base ensures that your fragrance will not mix with body odor or other unwanted scents, which could alter your chosen scent or weaken its potency.\n\nOnce your skin is clean, make sure it is completely dry before applying your fragrance. Wet or damp skin can dilute the scent, making it less effective and causing it to evaporate more quickly. Pat your skin dry with a towel and wait for a few minutes to ensure it is thoroughly dry before applying your cologne,

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [17]:
prompt = PromptTemplate(
    input_variables=["title","message","wordcount"],
    template="Write an article for Axe Deodrant with {title} that talks about the {message} and has a word count of minimum {wordcount}.\
            The article should have the following structure\
            Introduction : Give context to reader and introduce article theme. Use keywords from message\
            Post Shower Application : Discuss how fragrance should be applied on clean, dry, moisturized skin, why having clean, well-hydrated skin aids in retaining fragrance\
            Long-Lasting Fragrances : Discuss the importance of selecting fragrances with good longevity, include brief mention to the different types of fragrances and their staying power, additionally make reference to long-lasting fragrance notes\
            Layer Your Fragrances :  Discuss at large the concept of layering different fragrance products to enhance longevity, the benefits of using complimentary fragranced shower gels, lotions, and oils, as well as using deodorant and fragrance. Use Axe product examples such as Axe Blue Lavender Premium Deodorant Body Spray.\
            Spritz Don't Mist  ",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'title': "Number of Tips from Pros to Make Your Fragrance Last and Keep Your Scent Game Strong",
    'message': "In this listicle, we share top tips from Lil' Baby, sporting pros and \
                gamers on how they stay fresh AF all day by making their fragrance last.\
                Keywords to include: how to make cologne last longer, long-lasting deodorant , how \
                to make deodorant last longer , how to make fragrance last longer, long lasting deodorant spray,\
                how to layer fragrances, long lasting body spray, how long does body spray last,\
                how long do fragrances last, how to apply fragrance, how to make fragrance last\
                longer, where to apply fragrance, long lasting body spray for men, best fragrance \
                layering combinations, long lasting men's fragrance, deodorant ,\
                deodorant for men, body spray, antiperspirant, antiperspirant, \
                mens body spray, deodorant body spray",
    'wordcount': "4000"
    }))

 :  Discuss the concept of spritzing instead of misting and the importance of applying fragrance at a distance from the body, to avoid overwhelming the senses.            Strategically Target Application Sites :  Discuss the importance of applying and layering fragrances on pulse points and other areas to help boost scent.            Refresh Throughout the Day :  Discuss the importance of refreshing fragrances throughout the day, and the benefits of using long-lasting body sprays to refresh the scent. Use Axe product examples such as Axe Signature Gold Body Spray.            Conclusion : Summarize article and link to Axe product pages where readers can explore Axe fragrances.

Introduction
People who want to make sure their fragrance game is on point all day long know that it's not just about spritzing on your favorite scent and walking out the door. It's about making sure your fragrance lasts as long as possible so you can stay fresh and smell great all day. That's why we've gathered 

In [11]:
# Chain component 1: 

from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

template1 = """You will be given a message , title and wordcount from user. Generate an article based on the message,title and wordcount.Wordcount is the minimum word list of generated content.
    % MESSAGE 
    {message}
    % TITLE 
    {title}
    % WORDCOUNT 
    {wordcount}

YOUR RESPONSE:
"""
product_prompt = PromptTemplate(input_variables=["message","title","wordcount"], template=template1)

# LLMChain is a simple chain which takes a prompt template, formats with user input and returns LLM output.
product_chain = LLMChain(llm=llm, prompt=product_prompt)

In [12]:
# Chain component 2: 

template2 = """Given the message , title and wordcount, ensure you generate content with following topics.

% CONTENT
{content}

YOUR RESPONSE:
"""
product_content = PromptTemplate(input_variables=["content"], template=template2)

message_chain = LLMChain(llm=llm, prompt=product_content)

In [14]:
overall_chain = SimpleSequentialChain(
    chains=[product_chain, message_chain],
    verbose=True)
chain_output = overall_chain.run("In this listicle, we share top tips from Lil' Baby, \
                                 sporting pros and gamers on how they stay fresh AF all day by making their fragrance last.\
                            Keywords to include: how to make cologne last longer, long-lasting deodorant\
                                  , how to make deodorant last longer , how to make fragrance last longer,\
                                  long lasting deodorant spray, how to layer fragrances, long lasting body spray,\
                                  how long does body spray last, how long do fragrances last, how to apply\
                                  fragrance, how to make fragrance last longer, where to apply fragrance, long lasting\
                                  body spray for men, best fragrance layering combinations, long lasting men's fragrance,\
                                  deodorant , deodorant for men, body spray, antiperspirant, antiperspirant, mens body spray\
                                 , deodorant body spray")

ValidationError: 1 validation error for SimpleSequentialChain
__root__
  Chains used in SimplePipeline should all have one input, got memory=None callbacks=None callback_manager=None verbose=False tags=None prompt=PromptTemplate(input_variables=['message', 'title', 'wordcount'], output_parser=None, partial_variables={}, template='You will be given a message , title and wordcount from user. Generate an article based on the message,title and wordcount.Wordcount is the minimum word list of generated content.\n    % MESSAGE \n    {message}\n    % TITLE \n    {title}\n    % WORDCOUNT \n    {wordcount}\n\nYOUR RESPONSE:\n', template_format='f-string', validate_template=True) llm=AzureOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key='b5e76